**MAIN BODY OF DATA CLEANING**

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import pandas

import functions as f
import classes as c

In [14]:
df = f.getDataFrame("TrainOnMe.csv")
cleared_dataframe = f.clearData(df)[0]
#f.writeToCsv(cleared_dataframe, "ClearedSet.csv")

/Users/wiktorkowalczyk/Desktop/Personal/Code/github/machine_learning/Challenge/functions.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name] = pandas.to_numeric(dataframe[column_name], errors='coerce')
/Users/wiktorkowalczyk/Desktop/Personal/Code/github/machine_learning/Challenge/functions.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['x12'] = dataframe['x12'].astype(bool)


In [15]:
training_inputs, training_targets = f.extractData(cleared_dataframe)

In [16]:
# Get the training and testing data sets
X_train, X_test, y_train, y_test = train_test_split(training_inputs, training_targets, test_size=0.5)

**Boosted Decision Tree Classifier | NO CROSS VALIDATION**

In [17]:
pca = PCA(n_components=11)  # Adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Create an AdaBoostClassifier with a DecisionTree base estimator
base_estimator = DecisionTreeClassifier(max_depth=5)
ada_clf = AdaBoostClassifier(estimator=base_estimator, n_estimators=80)

# Fit the model to the training data
ada_clf.fit(X_train_pca, y_train)

# Make predictions on the test set
y_pred = ada_clf.predict(X_test_pca)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
accuracy_percent = accuracy * 100
print(f'Accuracy: {accuracy_percent:.2f}%')

Accuracy: 83.57%


**Classify the EvaluateOnMe.csv** |
**Boosted Decision Tree Classifier**

In [18]:
def testDTClassifier(training_inputs, training_targets, estimators=50, components=11, depth=5):
    X_train, X_test, y_train, y_test = train_test_split(training_inputs, training_targets, test_size=0.7)
    # Create an instance of the BoostedGaussianNaiveBayesClassifier
    boosted_dt_classifier = c.BoostedDecisionTreeClassifier(n_estimators=estimators, n_components=components, max_depth=depth)

    # Fit the classifier on the training data
    boosted_dt_classifier.fit(X_train, y_train)

    # Make predictions on the testing data
    predicted_labels = boosted_dt_classifier.predict(X_test)

    # Calculate the accuracy
    accuracy = accuracy_score(y_test, predicted_labels)
    accuracy *= 100
    return accuracy

In [19]:
# Sample data - replace this with your dataset
test_df = f.getDataFrame("TrainOnMe.csv")
test_cleared_dataframe, y_val, x_val = f.clearData(test_df)
test_training_inputs, test_training_targets = f.extractData(test_cleared_dataframe)

total_score = 0.0
for i in range(100):
    total_score += testDTClassifier(test_training_inputs, test_training_targets, estimators=80, components=12, depth=5)
print(f"""Total Accuracy: {total_score/100.0}""")

/Users/wiktorkowalczyk/Desktop/Personal/Code/github/machine_learning/Challenge/functions.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name] = pandas.to_numeric(dataframe[column_name], errors='coerce')
/Users/wiktorkowalczyk/Desktop/Personal/Code/github/machine_learning/Challenge/functions.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['x12'] = dataframe['x12'].astype(bool)


Total Accuracy: 82.50357653791133


In [20]:
# create the classifier object
decision_tree_classifier = c.BoostedDecisionTreeClassifier(n_estimators=80, max_depth=5, n_components=12)

# get the training data again from the file
df = f.getDataFrame("TrainOnMe.csv")
cleared_dataframe = f.clearData(df)[0]
#f.writeToCsv(cleared_dataframe, "ClearedSet.csv")
training_inputs, training_targets = f.extractData(cleared_dataframe)
# fit the classifier to the training data
decision_tree_classifier.fit(training_inputs, training_targets)

/Users/wiktorkowalczyk/Desktop/Personal/Code/github/machine_learning/Challenge/functions.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name] = pandas.to_numeric(dataframe[column_name], errors='coerce')
/Users/wiktorkowalczyk/Desktop/Personal/Code/github/machine_learning/Challenge/functions.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['x12'] = dataframe['x12'].astype(bool)


In [21]:
# create the classifier object
decision_tree_classifier = c.BoostedDecisionTreeClassifier(n_estimators=80, max_depth=5, n_components=12)

# get the training data again from the file
df = f.getDataFrame("TrainOnMe.csv")
cleared_dataframe, y_val, x_val = f.clearData(df)
#f.writeToCsv(cleared_dataframe, "ClearedSet.csv")
training_inputs, training_targets = f.extractData(cleared_dataframe)
# fit the classifier to the training data
decision_tree_classifier.fit(training_inputs, training_targets)

/Users/wiktorkowalczyk/Desktop/Personal/Code/github/machine_learning/Challenge/functions.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name] = pandas.to_numeric(dataframe[column_name], errors='coerce')
/Users/wiktorkowalczyk/Desktop/Personal/Code/github/machine_learning/Challenge/functions.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['x12'] = dataframe['x12'].astype(bool)


In [22]:
# predict the labels using the optimal boosted decision tree classifier
data_to_classify = f.getDataFrame("EvaluateOnMe.csv")
# handle data to classify: change the string values to make it possible to calssify
data_to_classify = f.clearDataToClassify(data_to_classify, x_val)
#writeToCsv(data_to_classify, "dataToClassifyCleared.csv")
inputs_to_classify = f.extractDataToClassify(data_to_classify)[0]

In [23]:
# predict the labels using the classifier
predictions = decision_tree_classifier.predict(inputs_to_classify)

In [24]:
# write the labels to a text file for grading
#f.write_labels_to_file(f.reverseTargetAssignment(predictions), "Labels.txt")